In [1]:
import pandas as pd
import operator 
import json
from collections import Counter
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
from nltk import bigrams 
import math
import numpy as np
import re
from mlxtend.preprocessing import one_hot
from gensim.models.doc2vec import LabeledSentence
from gensim.models import Doc2Vec
from gensim.models import word2vec 
from gensim.models import doc2vec
from gensim import utils
from random import shuffle
from sklearn.model_selection import train_test_split
import tensorflow as tf
import os
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function 
import argparse
import sys
from tensorflow.contrib.tensorboard.plugins import projector
FLAG =None
%matplotlib inline

/Users/haohanwang/anaconda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
data = pd.read_csv('tweets_with_emotion.csv',encoding = 'utf-8')

In [3]:
labels = data['Answer.sentiment']

lb = np.asarray(labels)
print(lb.shape)
sentence = data['Contents']
sentences = np.asarray(sentence)
# print(sentences[:5])

(9804,)


In [4]:
w = data['when']
# w1 = pd.get_dummies(w)
when = np.asarray(w)

print(when.shape)

(9804,)


In [5]:
y = np.asarray(data['NumLabel'], dtype = np.int32)
for i,val in enumerate(y):
    y[i] = y[i] -1    
y_ = one_hot(y, dtype='int')
y_ = y_.astype(np.float32)
# print(y_[:5])
# print(y_.dtype)

In [6]:

from __future__ import division
import gensim
from sklearn.decomposition import PCA
import itertools

def gensim_load_vec(path):
    gensim_emb = gensim.models.Word2Vec.load_word2vec_format(path, binary=False)
    vocab = gensim_emb.index2word
    vec =gensim_emb.syn0
    shape = gensim_emb.syn0.shape
    return gensim_emb, vec, shape, vocab

def map_word_frequency(document):
    return Counter(itertools.chain(*document))

def sentence2vec(tokenised_sentence_list, embedding_size, word_emb_model, a = 1e-3):
    word_counts = map_word_frequency(tokenised_sentence_list)
    sentence_set =[]
    sentence_words = []
    for sentence in tokenised_sentence_list:
        vs = np.zeros(embedding_size)
#         us = np.zeros(embedding_size)
        sentence_length = len(sentence)
        words =[]
        for word in sentence:
            a_value = a/(a+word_counts[word])
            words.append(word)
        try:
            vs = np.add(vs, np.multiply(a_value, word_emb_model[word]))
            
        except:
            pass
        vs = np.divide(vs, sentence_length)
        
        sentence_words.append([words[i:i+sentence_length] for i  in range(0, len(words), sentence_length)])
        sentence_set.append(vs)
        sent_words = np.squeeze(sentence_words,axis=0)
#         sentence_words.append(us)
        
    #calculate PCA of this sentence set
    pca = PCA(n_components= embedding_size)
    pca.fit(np.array(sentence_set))
    u = pca.explained_variance_ratio_
    u = np.multiply(u, np.transpose(u))
    
    if len(u) < embedding_size:
        for i in range(embedding_size -len(u)):
            u = np.append(u, 0)
            
    sentence_vecs = []
    
    for vs in sentence_set:
        sub = np.multiply(u,vs)
        sentence_vecs.append(np.subtract(vs, sub))
        sent_vecs = np.asarray(sentence_vecs)
        
    return sent_vecs,sent_words



In [7]:
# data = pd.read_csv('tweets_with_emotion.csv',encoding = 'utf-8')
tweets =[]
tweet= data['Contents'].tolist()
for i in tweet:    
    tweets.append(re.sub(r"http://\S+" and r"https://\S+", "", i))
t = [list(x.split()) for x in tweets]
print(len(t))

9804


In [8]:

import logging  
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality  
min_word_count = 1   # Minimum word count  
num_workers = 4       # Number of threads to run in parallel  
context = 4          # Context window size  
downsampling = 1e-3   # Downsample setting for frequent words

from gensim.models import word2vec  
print("Training model...")
model = word2vec.Word2Vec(t, workers=num_workers,  
            size=num_features, sg=1, min_count = min_word_count,
            window = context, sample = downsampling)


2018-01-06 14:26:06,974 : INFO : collecting all words and their counts
2018-01-06 14:26:06,975 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-01-06 14:26:07,022 : INFO : collected 20983 word types from a corpus of 174318 raw words and 9804 sentences
2018-01-06 14:26:07,023 : INFO : Loading a fresh vocabulary
2018-01-06 14:26:07,070 : INFO : min_count=1 retains 20983 unique words (100% of original 20983, drops 0)
2018-01-06 14:26:07,071 : INFO : min_count=1 leaves 174318 word corpus (100% of original 174318, drops 0)
2018-01-06 14:26:07,135 : INFO : deleting the raw counts dictionary of 20983 items
2018-01-06 14:26:07,137 : INFO : sample=0.001 downsamples 38 most-common words
2018-01-06 14:26:07,138 : INFO : downsampling leaves estimated 140666 word corpus (80.7% of prior 174318)
2018-01-06 14:26:07,139 : INFO : estimated required memory for 20983 words and 300 dimensions: 60850700 bytes


Training model...


2018-01-06 14:26:07,200 : INFO : resetting layer weights
2018-01-06 14:26:07,541 : INFO : training model with 4 workers on 20983 vocabulary and 300 features, using sg=1 hs=0 sample=0.001 negative=5 window=4
2018-01-06 14:26:08,555 : INFO : PROGRESS: at 60.76% examples, 424864 words/s, in_qsize 7, out_qsize 0
2018-01-06 14:26:09,105 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-01-06 14:26:09,124 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-01-06 14:26:09,134 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-01-06 14:26:09,152 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-01-06 14:26:09,153 : INFO : training on 871590 raw words (702941 effective words) took 1.6s, 438153 effective words/s


In [9]:
embedding_size = 300
sent_emb = sentence2vec(t, embedding_size, model)
# print(sent_emb)

In [10]:
emb_vec = sent_emb[0]
emb_vec = np.asarray(emb_vec)

In [11]:

sess = tf.InteractiveSession()
with tf.name_scope('input'):
    X = tf.placeholder(tf.float32, [None, 300], name ='x-input')
    Y = tf.placeholder(tf.float32, [None, 4],name = 'y-label')

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean',mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max_val', tf.reduce_max(var))
        tf.summary.scalar('min_val', tf.reduce_min(var))
        tf.summary.histogram('var_histogram',var)

def nn_layer(input_tensor, input_dim, output_dim, layer_name, act = tf.nn.relu):
    with tf.name_scope('Hidden_layer1'):

        with tf.name_scope('weights'):
            weights = weight_variable([input_dim, output_dim])
            variable_summaries(weights)
        with tf.name_scope('biases'):
            biases = bias_variable([output_dim])
            variable_summaries(biases)
        with tf.name_scope('output'):
            preactive = tf.matmul(input_tensor, weights)+ biases

        activations =act(preactive, name = 'output')
        tf.summary.histogram('output', activations)
        return activations

hidden1 = nn_layer(X, 300, 100, 'hidden_layer1')

with tf.name_scope('dropout'):
    keep_prob = tf.placeholder('float')
    tf.summary.scalar('dropout_keep_probability', keep_prob)
    dropped = tf.nn.dropout(hidden1, keep_prob)

y = nn_layer(dropped, 100, 4,'layer2', act=tf.nn.softmax)

with tf.name_scope('cross_entropy'):
    diff = tf.nn.softmax_cross_entropy_with_logits(logits = y, labels=Y)
    cross_entropy = tf.reduce_mean(diff)
tf.summary.scalar('cross_entropy', cross_entropy)

with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer(0.001).minimize(cross_entropy)

with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(Y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
tf.summary.scalar('accuracy', accuracy)



<tf.Tensor 'accuracy_1:0' shape=() dtype=string>

In [12]:
from textblob import TextBlob
model.vocab = model.wv.vocab
num_rows = len(sent_emb[1])
dim = model.vector_size

In [13]:
merged = tf.summary.merge_all()
train_writer = tf.summary.FileWriter('train',
                                    sess.graph)
test_writer = tf.summary.FileWriter('test')
w2v_10K = np.zeros((num_rows,dim))
vec =[]
with open("projector/projector.tsv", 'w+') as file_metadata:
    for i,word in enumerate(model.wv.index2word[:num_rows]):
        w2v_10K[i] = model[word]
        sentence = sentences[i]
        text_word = TextBlob(word)
        label = lb[i]
        time = when[i]

        if (i == 0):
            file_metadata.write('word'+'\t' +'when' +'\t' + 'label'+'\t'+'sentences'+'\n')    
        file_metadata.write(word + '\t' + str(time)+'\t' +str(label) +'\t'+ str(sentence)  +'\n')
        
sess = tf.InteractiveSession()

with tf.device('/cpu:0'):
    embedding = tf.Variable(w2v_10K, trainable=False, name='emotions_embedding')    
    
tf.global_variables_initializer().run()
saver = tf.train.Saver()
writer = tf.summary.FileWriter('projector',sess.graph)
config = projector.ProjectorConfig()
embed = config.embeddings.add()
embed.tensor_name = 'emotions_embedding'
embed.metadata_path = os.path.join('projector/projector.tsv')
projector.visualize_embeddings(writer, config)


'projector/emotion_embedding.ckpt-10000'

In [ ]:
sess = tf.InteractiveSession()
tf.global_variables_initializer().run(session = sess)
saver = tf.train.Saver()
config.model_checkpoint_path = os.path.join('projector','fashion.ckpt')
saver.save(sess, 'projector/emotion_embedding.ckpt', global_step=10000)

In [ ]:
size, num_features = train_images.shape
batch_size = 50
epochs = 1
num_iterations = size//batch_size
test_step = 500

for i in range(num_iterations*epochs):
    offset = (i*batch_size)% size
    batch_x = x_train[(offset):(offset+batch_size),:]
    batch_y = y_train[offset:(offset+batch_size),:]
    
    
    summary, _,loss = sess.run([merged, optimizer,cost],feed_dict={x: batch_x, y: batch_y})
    train_writer.add_summary(summary,i)
    if i % 100 == 0:
        
        print('Epoch {:<3} - Training Loss: {}'.format(i,loss))

